
## Input File Requirement to Generate Publications:
1. People ID (Row number recommended)
2. First Name
3. Last Name
4. Searched Term
5. Extra Information(Any Columns)

## Remember:
#####  The only needed variables are first 4. The rest variables will be kept as the input for later comparation.

## ReadMe!!!!, Update Description:
1. Now the input name and searched term are auto-collected to the ouput
2. Now it is a Full name Matching Algorithm!!!!!!!!
3. Latin Name, Letter Case Don't affect the Name Matching
4. A lot of useless columns droped.
5. Any amounts of Extra Information are available now!!!!!!!!!!!!!!


In [199]:
# import necessary libraries 
# Entrez is one library adapting the E-Utilities functions of Pubmed search for Python
# Note: no more than 3 queries/second using E-Utilities or more than 100,000 results / query -- will require setting up API Key from Pubmed
from Bio import Entrez
import pprint 
import numpy as np
import pandas as pd                 
from datetime import datetime
import os
import re     
import unidecode as un
#Register the API in PubMed!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Entrez.email = "juan.lin@h1insights.com"
Entrez.api_key = "9178358b4680ef4964a19857085715b84507"

In [200]:
# function to generate dataframe representing Pubmed search for one person with article information: Name, Organization Affiliation, Title of Paper, DOI 
def generator(PMID, record,shape):
    # initiate dictionaries for storing the different columns of information, will at end convert dictionary to series to pandas dataframe
    # dictionary adds additional security because keeps PMID value for the article information we're seeking
    dicts = {}
    dicts2 = {}
    dicts3 = {}
    dicts4 = {}
    title = []                           #article title - /ArticleTitle 
    CT = []                              #clinical trial - /DataBankList
    date = []                           #publication date - /Journal/PubDate
    peopleID = []                        #
    journal = []                        #journal title - /Journal/Title
    matchto = []
    #Conditional - Do not active!
    #org =[]  #Taiwan
    #inputname = []
    #department =[]
    #inputaffiliation = [] #China



    # iterates through the individual articles
    # length of pubmed articles in records (not PMID, because this will not include book articles (included in records at very end))
    for i in range(len(record['PubmedArticle'])):
        # initialize list for individual article doi, affiliation, name 
        peopleID.append(ID)
        matchto.append(info)
        #Conditional
       # inputname.append(inputname)
        #org.append(org)
        #department.append(department)
        #matchto.append(info)
       # inputaffiliation.append(inputaffiliation) #China
        
        
        doi = []                        #doi - /ELocationID & EIdType == "doi"
        affiliation = []
        mesh = []                       #Mesh -/MeshHeadingList
        name = []                       #Author Name - /AuthorList/ForeName + '' + /AuthorList/LastName
        # Titles of articles in list
        title.append(record['PubmedArticle'][i]['MedlineCitation']['Article']['ArticleTitle'])
        # Journal of which articles were published in 
        journal.append(record['PubmedArticle'][i]['MedlineCitation']['Article']['Journal']['Title'])
        # Existence of clinical trial link in the Article                                                                                    
        if 'DataBankList' in record['PubmedArticle'][i]['MedlineCitation']['Article']:
            CT.append(record['PubmedArticle'][i]['MedlineCitation']['Article']['DataBankList'][0]['AccessionNumberList'][0])
        else: 
            CT.append("NA")
        # Publication date of paper
        if 'PubDate' in record['PubmedArticle'][i]['MedlineCitation']['Article']['Journal']['JournalIssue']:
            if 'Year' in record['PubmedArticle'][i]['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']:
                if 'Month' in record['PubmedArticle'][i]['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']:
                    year = record['PubmedArticle'][i]['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Year']
                    month = record['PubmedArticle'][i]['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Month']
                    fulldate = month + ' ' + year
                    date.append(fulldate)
                else: 
                    date.append(record['PubmedArticle'][i]['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Year'])     
            else: 
                date.append("NA")                                                                                                           
        for j in range(len(record['PubmedArticle'][i]['MedlineCitation']['Article']['ELocationID'])):
            strings = record['PubmedArticle'][i]['MedlineCitation']['Article']['ELocationID'][j]
            # searches and selects for ID type that is DOI (PII also found here)
            if strings.attributes['EIdType'] == "doi": 
                doi.append(strings)
        # one PMID, one DOI 
        dicts[PMID[i]] = doi
        # iterates through author list to search for correct person and affiliation
        for k in range(len(record['PubmedArticle'][i]['MedlineCitation']['Article']['AuthorList'])):                                    
            authors = record['PubmedArticle'][i]['MedlineCitation']['Article']['AuthorList'][k]
            # if author has an author affiliation
            if authors['AffiliationInfo'] != [] and 'LastName' and 'ForeName' in authors:               
                # match on first and last name 
                    #if un.unidecode(authors['LastName']).upper() == un.unidecode(authorlast.upper()) and un.unidecode(authors['ForeName'].upper()) == un.unidecode(authorfirst.upper()):  
                    if un.unidecode(authors['ForeName'].upper())+' '+un.unidecode(authors['LastName']).upper()==un.unidecode(authorfirst.upper())+' '+un.unidecode(authorlast.upper()):    
                        firstname = authors['ForeName']
                        lastname = authors['LastName']
                        fullname = firstname + ' ' + lastname
                        name.append(fullname)
                    # collect all affiliations attributed to the matched author
                        for l in range(len(authors['AffiliationInfo'])):
                            affiliation.append(authors['AffiliationInfo'][l]['Affiliation'])
            # in the cases for which author affiliation is blank, assign to the affiliation of the first   
            #elif 'LastName' and 'ForeName' in authors and un.unidecode(authors['LastName'].upper()) == un.unidecode(authorlast.upper()) and un.unidecode(authors['ForeName'].upper()) == un.unidecode(authorfirst.upper()):
            elif 'LastName' and 'ForeName' in authors and un.unidecode(authors['ForeName'].upper())+' '+un.unidecode(authors['LastName']).upper()==un.unidecode(authorfirst.upper())+' '+un.unidecode(authorlast.upper()): 
                if record['PubmedArticle'][i]['MedlineCitation']['Article']['AuthorList'][0]['AffiliationInfo'] != []:
                    firstname = authors['ForeName']
                    lastname = authors['LastName']
                    fullname = firstname + ' ' + lastname
                    name.append(fullname)
                    for l in range(len(record['PubmedArticle'][i]['MedlineCitation']['Article']['AuthorList'][0]['AffiliationInfo'])):
                        conditionalaff = "FIRST AUTHOR AFFILIATION: " + record['PubmedArticle'][i]['MedlineCitation']['Article']['AuthorList'][0]['AffiliationInfo'][l]['Affiliation']
                        affiliation.append(conditionalaff)
             #   elif record['PubmedArticle'][i]['MedlineCitation']['Article']['AuthorList'][-1]['AffiliationInfo'] != []:
            #        firstname = authors['ForeName']
             #       lastname = authors['LastName']
             #       fullname = firstname + ' ' + lastname
             #       name.append(fullname)
             #       for l in range(len(record['PubmedArticle'][i]['MedlineCitation']['Article']['AuthorList'][-1]['AffiliationInfo'])): #NEW
              #          conditionalaff = "LAST AUTHOR AFFILIATION: " + record['PubmedArticle'][i]['MedlineCitation']['Article']['AuthorList'][0]['AffiliationInfo'][l]['Affiliation'] #NEW
              #          affiliation.append(conditionalaff) #NEW
              #          

        # iterate through mesh headings 
        if 'MeshHeadingList' in record['PubmedArticle'][i]['MedlineCitation']:
            for m in range(len(record['PubmedArticle'][i]['MedlineCitation']['MeshHeadingList'])): 
                # [:] allows for getting all of the inside (no string element values) when exporting the dataframe into excel 
                keywords = record['PubmedArticle'][i]['MedlineCitation']['MeshHeadingList'][m]['DescriptorName'][:]
                mesh.append(keywords)
            
        dicts2[PMID[i]] = affiliation
        dicts3[PMID[i]] = name
        dicts4[PMID[i]] = mesh
        
    # dictionary to series to dataframe, named columns
    dicts = pd.Series(dicts).to_frame()                                                                                                                               
    dicts.columns = ['DOI']
    dicts2 = pd.Series(dicts2).to_frame()
    dicts2.columns = ['Affiliation']
    dicts2.Affiliation.apply(tuple)                                                                                                                                    
    dicts3 = pd.Series(dicts3).to_frame()
    dicts3.columns = ['Name']
    dicts4 = pd.Series(dicts4).to_frame()
    dicts4.columns = ['Mesh']    
    
    #join into one dataframe, nice formatting
    publications = dicts2.join(dicts3)
    #publications = dicts2.join(dicts3)
    #publications.DOI = publications.DOI.str[0]
    #publications.Name = publications.Name.str[0]
    publications['Title'] = title
    #publications['Clinical Trials'] = CT
    #publications['Date'] = date
    publications['PeopleID'] = peopleID
    #publications['Journal'] = journal
    publications['Searched Term'] = matchto
    #Conditional
    #publications['Department'] = department 
    #publications['Input Affiliation'] = org   #Taiwan
    #publications['Input Affiliation'] = inputaffiliation  #China
    publications['Input Name'] = inputname
    #China
    #publications['Input Affiliation'] = inputname
    
    #publications = publications.join(dicts4)
    if shape>4:
        for i in range(5,shape+1):
            exec('publications[{0}Extra_{1}{0}] = extra_{2}'.format("'",i-4,i-4))    

    return publications


----
# CASE 1: Search with search terms

Set your directory here:

In [208]:
# Set your directory
os.chdir('C:/Users/Juan/Desktop/ipython')
df = pd.read_excel("test.xlsx", index_col = None)  

In [209]:
# pd.set_option('max_colwidth', 300)
pd.set_option('display.max_rows', 6)
df

,3,IMAM,SUBEKTI,CILILIN,"CILILIN, RSUD",Pediatrics,Pediatrics.1,ENDO_Endocrinologist Paediatric,Endocrinology,JL.CINTA KARYA DESA CILILIN CILILIN - CILILIN,Unnamed: 10,40562,CILILIN
0,4,BUDI,WIWEKO,SENEN,"CIPTO MANGUNKUSUMO, RSUPN",Obstetrician Gynecologist,Reproductive Endocrinology Infertility Consultant,FERT_Fertility Focused,Fertility,JL. DIPONEGORO NO. 71 KENARI - SENEN,NaN,10430,SENEN
1,6,MADARINA,JULIA,GONDOKUSUMAN,"DR. SARDJITO, RSUP",Pediatrics,Pediatric Endocrinologist,ENDO_Endocrinologist Paediatric,Endocrinology,JL.IPDA TUT HARSONO NO.53 TIMOHO BACIRO-GONDOK...,NaN,55225,GONDOKUSUMAN
2,21,ANDON,HESTIANTORO,SENEN,"CIPTO MANGUNKUSUMO, RSUPN",Obstetrician Gynecologist,Reproductive Endocrinology Infertility Consultant,FERT_Fertility Focused,Fertility,JL. DIPONEGORO NO. 71 KENARI - SENEN,NaN,10430,SENEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,115,IVAN RIZAL,SINI,MENTENG,"BUNDA MEDIK, RS / MENTENG",Obstetrician Gynecologist,Obstetrician Gynecologist,FERT_Fertility Generalist,Fertility,JL. TEUKU CIK DITIRO NO. 28 GONDANGDIA - MENTENG,NaN,14440,MENTENG
24,141,DHARMA,DHARMA,SENEN,"CIPTO MANGUNKUSUMO, RSUPN",Neurology,Neurology,NEURO_Neurologist Rx MS,Neurology,JL. DIPONEGORO NO. 71 KENARI - SENEN,NaN,10430,SENEN
25,160,ARRY,REKSODIPUTRO,SENEN,"CIPTO MANGUNKUSUMO, RSUPN",Internal Medicine,Hematology Oncology,ONC_Colorectal Cancer Specialist,Oncology,JL. DIPONEGORO NO. 71 KENARI - SENEN,NaN,10430,SENEN


In [210]:
publications=pd.core.frame.DataFrame()
import time
start=time.time()
placeholder = []
noresults1 = []
noresults2 = []
noresults3 = []
shape=df.shape[1]
for row in df.itertuples(): 
    ID = row[1]
    authorfirst = row[2]
    authorlast = row[3]
    org = row[4]   #Taiwan
    info = row[4]
    inputname = row[2]+' '+row[3]
    #extra
    if shape>4:
        for i in range(5,shape+1):
            exec('extra_{}=row[{}]'.format(i-4,i))
    
    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    ## set search 
    #search = '({} {}[Author]) AND ({}[Affiliation])'.format(authorfirst, authorlast, org)
    #search = '{} {} {}'.format(authorfirst, authorlast, org)
    search = '{} {}[Author]'.format(authorfirst, authorlast)
    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    
    
    
    # E-utiltiies esearch searches pubmed, returns max of 500 articles
    handle = Entrez.esearch(db = "pubmed", term = search, retmax = 5)  #set retmax = 5 for Taiwan
    # read parses and returns in simplified format
    record = Entrez.read(handle)
    # in the case that search results exist
               # in the case that search results exist
    if 'ErrorList' in record: 
        noresults1.append(ID)
        noresults2.append(info)
        noresults3.append("No Relevant Results")
        
    elif record['IdList'] != []:
        # this is list of PMIDs from search
        PMID = record['IdList']  
        # input PMID list of articles into efetch, returns XML of articles
        handle = Entrez.efetch(db="pubmed", id=PMID, retmode="xml")
        # parses accordingly
        record2 = Entrez.read(handle)
        # run function to get list of dataframes (one dataframe for each person), concat dataframe together
        publication = generator(PMID, record2,shape)
        placeholder.append(publication)
    else: 
        noresults1.append(ID)
        noresults2.append(info)
        noresults3.append("No Results")
publications = pd.concat(placeholder)
end=time.time()
run_time=end-start
run_time

23.16836643218994

In [211]:
publications

,Affiliation,Name,Title,PeopleID,Searched Term,Input Name,Extra_1,Extra_2,Extra_3,Extra_4,Extra_5,Extra_6,Extra_7,Extra_8,Extra_9
33838481,[Division of Reproductive Endocrinology and In...,[Budi Wiweko],Early-onset complete spontaneous migration of ...,4,SENEN,BUDI WIWEKO,"CIPTO MANGUNKUSUMO, RSUPN",Obstetrician Gynecologist,Reproductive Endocrinology Infertility Consultant,FERT_Fertility Focused,Fertility,JL. DIPONEGORO NO. 71 KENARI - SENEN,NaN,10430,SENEN
33747776,"[Department of Obstetrics and Gynecology, Repr...",[Budi Wiweko],Heterotopic Pregnancy: Diagnosis and Pitfall i...,4,SENEN,BUDI WIWEKO,"CIPTO MANGUNKUSUMO, RSUPN",Obstetrician Gynecologist,Reproductive Endocrinology Infertility Consultant,FERT_Fertility Focused,Fertility,JL. DIPONEGORO NO. 71 KENARI - SENEN,NaN,10430,SENEN
33639673,"[Department of Obstetrics and Gynecology, Facu...",[Budi Wiweko],Histopathology and ARID1A Expression in Endome...,4,SENEN,BUDI WIWEKO,"CIPTO MANGUNKUSUMO, RSUPN",Obstetrician Gynecologist,Reproductive Endocrinology Infertility Consultant,FERT_Fertility Focused,Fertility,JL. DIPONEGORO NO. 71 KENARI - SENEN,NaN,10430,SENEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28939746,[],[],Phase III Clinical Trial (RERISE study) Result...,160,SENEN,ARRY REKSODIPUTRO,"CIPTO MANGUNKUSUMO, RSUPN",Internal Medicine,Hematology Oncology,ONC_Colorectal Cancer Specialist,Oncology,JL. DIPONEGORO NO. 71 KENARI - SENEN,NaN,10430,SENEN
23910494,[],[],Hepatitis B virus reactivation in B-cell lymph...,160,SENEN,ARRY REKSODIPUTRO,"CIPTO MANGUNKUSUMO, RSUPN",Internal Medicine,Hematology Oncology,ONC_Colorectal Cancer Specialist,Oncology,JL. DIPONEGORO NO. 71 KENARI - SENEN,NaN,10430,SENEN
22273250,[],[],Non-bacterial infections in Asian patients tre...,160,SENEN,ARRY REKSODIPUTRO,"CIPTO MANGUNKUSUMO, RSUPN",Internal Medicine,Hematology Oncology,ONC_Colorectal Cancer Specialist,Oncology,JL. DIPONEGORO NO. 71 KENARI - SENEN,NaN,10430,SENEN


In [212]:
noresults = pd.DataFrame(
    {'No Results': noresults1,
     'Search Term': noresults2,
     'Reason': noresults3
    })
pprint.pprint(noresults) 
# print(noresults2)

noresults.to_excel("Noresults with Missing Name.xlsx")

#now = datetime.now()
#date_time = now.strftime("%Y%m%d")


file_name = "EU5_2"


publications.to_excel(f'{file_name}_Output1.xlsx')
#De empty
#Output1=pd.read_excel(f'{file_name}_Output1.xlsx', index_col = None)  
#Output1_clear=Output1.drop(Output1[Output1['Name']=='[]'].index)
#Output1_clear.to_excel(f'{file_name}_Output2.xlsx')
#noresults_2 = pd.DataFrame(
#    {'No Results':list(set(list(Output1['PeopleID'][Output1[Output1['Name']=='[]'].index])+noresults1))
#    })
#pprint.pprint(noresults_2)
#noresults_2.to_excel(f'{file_name}Noresults.xlsx')

Empty DataFrame
Columns: [No Results, Search Term, Reason]
Index: []


In [213]:
Output1_clear

,Unnamed: 0,Affiliation,Name,Title,PeopleID,Match To,Input Name
0,31659852,"['Endocrinology Division, Child Health Departm...",['Aman Bhakti Pulungan'],ISPAD Clinical Practice Consensus Guidelines: ...,2,Pediatrics,AMANBHAKTI PULUNGAN
1,33833872,"['Department of Internal Medicine, Faculty of ...",['Ketut Suastika'],Association of Apolipoprotein E Gene Polymorph...,5,Internal Medicine,KETUTSUASTIKA
2,33395778,"['Head, Division of Endocrinology and Metaboli...",['Ketut Suastika'],The Effect of Metformin Consumption on Mortali...,5,Internal Medicine,KETUTSUASTIKA
...,...,...,...,...,...,...,...
157,32047682,"['Division of Hematology-Medical Oncology, Dep...",['Cosphiadi Irawan'],Undifferentiated Nasopharyngeal Carcinoma Pres...,162,Internal Medicine,COSPHIADIIRAWAN
158,32041912,['FIRST AUTHOR AFFILIATION: Division of Psycho...,['Cosphiadi Irawan'],Factors Associated with Pain in Palliative Pat...,162,Internal Medicine,COSPHIADIIRAWAN
159,31383830,['FIRST AUTHOR AFFILIATION: Division of Psycho...,['Cosphiadi Irawan'],"Characteristics of Palliative Patients, Insigh...",162,Internal Medicine,COSPHIADIIRAWAN
